In [98]:
#pip install openai

# 1. AI zero sample reasoning for ob paper screening

In [99]:
import os
import pandas as pd
import openai
import time
#from deepseek import DeepSeek
from tqdm import tqdm  # time bar
from openai import OpenAI

## 1.1 Call api from siliconflow

可在DS/openai官网创建api key并充值，免费可用gemini（有次数限制）、硅基流动(silicon.flow)上一些免费llm模型or赠金、或者下载lamma等本地运行

In [ ]:
url = 'https://*****' #如url = 'https://api.siliconflow.cn/v1/'， "https://api.deepseek.com"等
api_key = 'sk-*******' #your api_key

client = OpenAI(
    base_url=url,
    api_key=api_key
)

## 1.2 Test

In [ ]:
# example_try 非流式请求

'''
prompt = f"""Please evaluate whether the following paper belongs to, is highly relevant to, or has significant implications for the field of Organizational Behavior (OB) or Human Resource Management (HRM).

Please exclude if primary focus:
• Macroeconomics, finance, strategic management 
  (market entry, firm-level strategy, M&A)
• Basic psychology without organizational application:
  - Developmental/cognitive/clinical psychology  
  - Neuroscience, perception, memory studies
• Consumer behavior/behavioral economics without workplace application
• Education/public health not linked to organizations
• Legal studies/political science/criminology (unless workplace-focused)
• Purely technological/computational models without behavioral insights

Otherwise, the standard can be more lenient; as long as the paper has some relevance or might provide psychological insights to OB/HRM, it should be included.

Please respond with only "Yes" or "No" and do not provide any additional information.
Title: {"Time is fundamental to organizing all aspects"}
Abstract: {"Time is fundamental to organizing all aspects"}
Keyword: {"Time is fundamental to organizing all aspects"}
"""

# 发送非流式输出请求
messages = [
    {"role": "user", "content": prompt}
]
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-0528-Qwen3-8B",
    messages=messages,
    stream=False, 
    max_tokens=4000
)
content = response.choices[0].message.content
reasoning_content = response.choices[0].message.reasoning_content

print(content)

'''

'\nprompt = f"""Please evaluate whether the following paper belongs to, is highly relevant to, or has significant implications for the field of Organizational Behavior (OB) or Human Resource Management (HRM).\n\nPlease exclude if primary focus:\n• Macroeconomics, finance, strategic management \n  (market entry, firm-level strategy, M&A)\n• Basic psychology without organizational application:\n  - Developmental/cognitive/clinical psychology  \n  - Neuroscience, perception, memory studies\n• Consumer behavior/behavioral economics without workplace application\n• Education/public health not linked to organizations\n• Legal studies/political science/criminology (unless workplace-focused)\n• Purely technological/computational models without behavioral insights\n\nOtherwise, the standard can be more lenient; as long as the paper has some relevance or might provide psychological insights to OB/HRM, it should be included.\n\nPlease respond with only "Yes" or "No" and do not provide any additio

In [ ]:
# 流式请求示例
"""

# 发送带有流式输出的请求
content = ""
reasoning_content=""
messages = [
    {"role": "user", "content": "Yichen said hello to you, please start a small talk with her politely"}
]
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1",
    messages=messages,
    stream=True,  # 启用流式输出
    max_tokens=4096
)
# 逐步接收并处理响应
for chunk in response:
    if chunk.choices[0].delta.content:
        content += chunk.choices[0].delta.content
    if chunk.choices[0].delta.reasoning_content:
        reasoning_content += chunk.choices[0].delta.reasoning_content

# Round 2
messages.append({"role": "assistant", "content": content})
messages.append({'role': 'user', 'content': "继续"})
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1",
    messages=messages,
    stream=True
)
"""

'\nurl = \'https://api.siliconflow.cn/v1/\'\napi_key = \'your api_key\'\n\nclient = OpenAI(\n    base_url=url,\n    api_key=api_key\n)\n\n# 发送带有流式输出的请求\ncontent = ""\nreasoning_content=""\nmessages = [\n    {"role": "user", "content": "奥运会的传奇名将有哪些？"}\n]\nresponse = client.chat.completions.create(\n    model="deepseek-ai/DeepSeek-R1",\n    messages=messages,\n    stream=True,  # 启用流式输出\n    max_tokens=4096\n)\n# 逐步接收并处理响应\nfor chunk in response:\n    if chunk.choices[0].delta.content:\n        content += chunk.choices[0].delta.content\n    if chunk.choices[0].delta.reasoning_content:\n        reasoning_content += chunk.choices[0].delta.reasoning_content\n\n# Round 2\nmessages.append({"role": "assistant", "content": content})\nmessages.append({\'role\': \'user\', \'content\': "继续"})\nresponse = client.chat.completions.create(\n    model="deepseek-ai/DeepSeek-R1",\n    messages=messages,\n    stream=True\n)\n'

# 2. Define Functions

## 2.1 Function for screening

可以修改prompt为相应学科/主题文献筛选要求
可以加\[例子\]

In [103]:
# function for screening OB paper
def is_ob_paper(title, abstract):
    prompt = f"""
Please evaluate whether the following paper belongs to, is highly relevant to, or has significant implications for the field of Organizational Behavior (OB) or Human Resource Management (HRM).

Please exclude if primary focus:
• Macroeconomics, finance, strategic management 
  (market entry, firm-level strategy, M&A)
• Basic psychology lacking organizational application:
  - Developmental/cognitive/clinical/personality/educational psychology  
  - Neuroscience, perception, memory studies
• Consumer behavior/behavioral economics without workplace application
• Education/public health not linked to organizations
• Legal studies/political science/criminology (unless workplace-focused)
• Purely technological/computational models without behavioral insights

Otherwise, the standard can be more lenient; as long as the paper has some relevance or might provide psychological insights to OB/HRM, it should be included.

Please respond with only "Yes" or "No" and do not provide any additional information.

Title: {title}
Abstract: {abstract}

"""
    try:
        response = client.chat.completions.create(
            model="deepseek-ai/DeepSeek-R1-0528-Qwen3-8B",  # 使用实际模型名
            #model = "Qwen/Qwen3-8B",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,  # 固定输出减少幻觉
            max_tokens=5,
            stream=False   # 关闭流式输出
            #enable_thinking=False  # 禁用思考内容（如平台支持）
        )
        
        answer = response.choices[0].message.content.strip()
        return 1 if answer == "Yes" else 0
    except Exception as e:
        print(f"API call error: {e}")
        return -1

## 2.2 Function for Extraction

同理，可修改prompt调整输出内容

In [122]:

# function to extract information using AI prompt

def extract_lit_elements(title, abstract):
    prompt = f"""
I want to conduct a literature review. Please extract the following elements from the abstract, if available. Use only one or two sentences for each element:

- Research question:
- Finding Summary:
- Theory:
- IV:
- Mediator:
- DV:
- Moderator:

Abstract: {abstract}
"""
    try:
        response = client.chat.completions.create(
            #model="deepseek-ai/DeepSeek-R1-0528-Qwen3-8B",  # 或改为你用的模型名
            model = "Qwen/Qwen3-8B",
            #enable_thinking = False,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=512,
            stream=False
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"API call error: {e}")
        return ""


# 3. Task Processing

## 3.1 data loading

In [ ]:
# data loading
input_path = "JPSP3536_Yichen.csv"  
output_path = "JPSP3536_Yichen_100_1.csv"

df = pd.read_csv(input_path)
test_df = df.iloc[0:100].copy()  # 选200行作为sample

## 3.2 Screening

In [ ]:
results = []
total_rows = len(test_df)  # 数据量
failed_count = 0

print(f"开始处理 {total_rows} 篇论文(测试模式)...")
for idx, row in tqdm(test_df.iterrows(), total=total_rows):
    title = row.get("Title", "")
    abstract = row.get("Abstract", "")
    #keywords = row.get("Keyword", "")
    
    # 检查缺失值
    if pd.isna(title) or pd.isna(abstract):  #如果有缺失abstract记得修改这里逻辑
        results.append(0)
        continue
    
    try:
        ob_flag = is_ob_paper(title, abstract)
        results.append(ob_flag)
    except Exception as e:
        print(f"\n处理第{idx+1}行时出错: {str(e)}")
        results.append(0)
        failed_count += 1
    
    # 频率控制 (每秒1次请求 + 每100次额外休息)
    time.sleep(1)  # 基础延迟
    if (idx + 1) % 100 == 0:
        time.sleep(5)  # 每100次额外休息5秒

# save results
test_df["OB"] = results


# print outcome info
success_count = total_rows - failed_count
print(f"\n处理完成! 成功: {success_count}/{total_rows} | 失败: {failed_count}")
#print(f"结果已保存到: {output_path}")
print("OB paper in total:")
print(df["OB"].value_counts())
test_df.head()

开始处理 100 篇论文(测试模式)...


100%|██████████| 100/100 [2:00:23<00:00, 72.24s/it]  



处理完成! 成功: 100/100 | 失败: 0
OB paper in total:
Series([], Name: count, dtype: int64)


,Author Full Name,Title,Journal,Year,Abstract,Keyword,DOI,OB,Research question,Finding Summary,Theory,IV,Mediator,DV,Moderator
0,"Srivastava, S; John, OP; Gosling, SD; Potter, J",Development of personality in early and middle...,JOURNAL OF PERSONALITY AND SOCIAL PSYCHOLOGY,2003,Different theories make different predictions ...,NaN,10.1037/0022-3514.84.5.1041,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Anderson, C; Keltner, D; John, OP",Emotional convergence between people over time,JOURNAL OF PERSONALITY AND SOCIAL PSYCHOLOGY,2003,The authors propose that people in relationshi...,NaN,10.1037/0022-3514.84.5.1054,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Compton, RJ; Fisher, LR; Koenig, LM; McKeown, ...",Relationship between coping styles and percept...,JOURNAL OF PERSONALITY AND SOCIAL PSYCHOLOGY,2003,This study examined the relationship between c...,NaN,10.1037/0022-3514.84.5.1069,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Sellers, RM; Shelton, JN",The role of racial identity in perceived racia...,JOURNAL OF PERSONALITY AND SOCIAL PSYCHOLOGY,2003,This study examined the role that dimensions o...,NaN,10.1037/0022-3514.84.5.1079,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"van Baaren, RB; Maddux, WW; Chartrand, TL; de ...",It takes two to mimic: Behavioral consequences...,JOURNAL OF PERSONALITY AND SOCIAL PSYCHOLOGY,2003,The present studies demonstrated the moderatio...,NaN,10.1037/0022-3514.84.5.1093,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3.3 Extraction

In [123]:
# extraction

# 仅保留OB文章
filtered_df = test_df[test_df['OB'] == 1]
results = []
total_rows = len(filtered_df)
failed_count = 0


print(f"开始处理 {total_rows} 篇论文(测试模式)...")
for idx, row in tqdm(filtered_df.iterrows(), total=total_rows):
    title = row.get("Title", "")
    abstract = row.get("Abstract", "")
    #keywords = row.get("Keyword", "")

    if pd.isna(title) or pd.isna(abstract):
        results.append("")
        continue

    try:
        extraction = extract_lit_elements(title, abstract)
        results.append(extraction)
    except Exception as e:
        print(f"\n处理第{idx+1}行时出错: {str(e)}")
        results.append("")
        failed_count += 1

    # 请求频率控制
    time.sleep(1)
    if (idx + 1) % 100 == 0:
        time.sleep(5)

# ===== 保存结果列 =====
filtered_df["Extracted"] = results

# 输出处理信息
success_count = total_rows - failed_count
print(f"\n处理完成! 成功: {success_count}/{total_rows} | 失败: {failed_count}")

开始处理 85 篇论文(测试模式)...


100%|██████████| 85/85 [1:30:23<00:00, 63.81s/it]   


处理完成! 成功: 85/85 | 失败: 0



/var/folders/fw/_5f21xzd68b7405zhp2xqtr00000gn/T/ipykernel_9598/2409672980.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["Extracted"] = results


# 4. Output

## 4.1 data format


In [ ]:
import re

def extract_fields(text):
    """
    Extract multiple predefined fields from structured text using a robust regex.
    Returns a dictionary mapping field names to cleaned values.
    """
    if not isinstance(text, str):
        return {field: "" for field in fields}

    # regex pattern, modify or choose the right one according to Extracted output
    #pattern = r"-\s*\*\*(.+?)\*\*\s*[:：]\s*(.*?)(?=\n\s*-\s*\*\*|$)"
    #pattern = r"\*\s*\*\*(.+?)[:：]?\*\*\s*[:：]?\s*(.*?)(?=\n\s*\*\s*\*\*|$)"
    pattern = r"-\s*\*\*(.+?)[:：]?\*\*\s*[:：]?\s*(.*?)(?=\n\s*-\s*\*\*|$)"

    
    #  DOTALL - match multiline field values
    matches = re.findall(pattern, text, re.DOTALL)

    result = {}
    for key, value in matches:
        cleaned_key = key.strip()
        cleaned_value = re.sub(r'\s+', ' ', value).strip()  
        result[cleaned_key] = cleaned_value

    # ensure all expected columns are present
    return {field: result.get(field, "") for field in fields}


# expected columns (fileds)
fields = ["Research question", "Finding Summary", "Theory", "IV", "Mediator", "DV", "Moderator"]

# execute in sample df:
for field in fields:
    filtered_df[field] = filtered_df["Extracted"].apply(lambda x: extract_fields(x).get(field, ""))

/var/folders/fw/_5f21xzd68b7405zhp2xqtr00000gn/T/ipykernel_9598/1197621437.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[field] = filtered_df["Extracted"].apply(lambda x: extract_fields(x).get(field, ""))


In [126]:
filtered_df.iloc[1]['Extracted']

"- **Research question**: How does emotional similarity develop in relationships over time, and what are its effects on relationship cohesion and stability?  \n- **Finding Summary**: Over a year, dating partners and college roommates showed increased emotional similarity, with those in less powerful relationships exhibiting greater change toward convergence.  \n- **Theory**: The theory of emotional convergence posits that emotional similarity enhances coordination, understanding, and social cohesion in relationships.  \n- **IV**: The independent variable is the duration of the relationship (a year) and the type of relationship (dating partners vs. college roommates).  \n- **Mediator**: Emotional similarity mediates the relationship between partners' interactions and the development of social cohesion.  \n- **DV**: The dependent variables are relationship cohesion and the likelihood of dissolution.  \n- **Moderator**: Power dynamics moderate the extent of emotional convergence, with les

In [127]:
filtered_df.iloc[2]['Research question']

'How do coping styles, particularly ruminative coping, relate to hemispheric asymmetry and depression?'

In [128]:
filtered_df.head()

,Author Full Name,Title,Journal,Year,Abstract,Keyword,DOI,OB,Research question,Finding Summary,Theory,IV,Mediator,DV,Moderator,Extracted
0,"Srivastava, S; John, OP; Gosling, SD; Potter, J",Development of personality in early and middle...,JOURNAL OF PERSONALITY AND SOCIAL PSYCHOLOGY,2003,Different theories make different predictions ...,NaN,10.1037/0022-3514.84.5.1041,1,How do different theories (biological vs. cont...,The study found that Conscientiousness and Agr...,The biological view (plaster hypothesis) posit...,"Age (21–60 years) as the independent variable,...","Developmental influences, as the findings high...",Mean levels of Big Five personality traits (Co...,"Gender, as the effect of Neuroticism change di...",- **Research question**: How do different theo...
1,"Anderson, C; Keltner, D; John, OP",Emotional convergence between people over time,JOURNAL OF PERSONALITY AND SOCIAL PSYCHOLOGY,2003,The authors propose that people in relationshi...,NaN,10.1037/0022-3514.84.5.1054,1,How does emotional similarity develop in relat...,"Over a year, dating partners and college roomm...",The theory of emotional convergence posits tha...,The independent variable is the duration of th...,Emotional similarity mediates the relationship...,The dependent variables are relationship cohes...,Power dynamics moderate the extent of emotiona...,- **Research question**: How does emotional si...
2,"Compton, RJ; Fisher, LR; Koenig, LM; McKeown, ...",Relationship between coping styles and percept...,JOURNAL OF PERSONALITY AND SOCIAL PSYCHOLOGY,2003,This study examined the relationship between c...,NaN,10.1037/0022-3514.84.5.1069,1,"How do coping styles, particularly ruminative ...","In women, self-reported rumination was linked ...",The study is grounded in the theory that reduc...,"Coping styles, measured via self-report (Respo...",Hemispheric asymmetry (RH bias) mediates the r...,,Gender moderates the effect of rumination on R...,"- **Research question**: How do coping styles,..."
3,"Sellers, RM; Shelton, JN",The role of racial identity in perceived racia...,JOURNAL OF PERSONALITY AND SOCIAL PSYCHOLOGY,2003,This study examined the role that dimensions o...,NaN,10.1037/0022-3514.84.5.1079,1,How do dimensions of racial identity influence...,Racial centrality predicted higher perceived d...,"Racial identity theory, which posits that raci...",Racial identity dimensions (specifically racia...,Not explicitly mentioned in the abstract.,Perceived racial discrimination and psychologi...,Racial ideology and public regard beliefs.,- **Research question**: How do dimensions of ...
5,"Shah, J",Automatic for the people: How representations ...,JOURNAL OF PERSONALITY AND SOCIAL PSYCHOLOGY,2003,Five studies are presented that explore how re...,NaN,10.1037/0022-3514.84.4.661,1,How do representations of significant others a...,Goals are primed by representations of signifi...,"The study builds on social influence theory, e...","Representations of significant others (e.g., t...",Goal priming (activation of goals through asso...,Task persistence and performance (as indicator...,Closeness to the significant other and the num...,- **Research question**: How do representation...


## 4.2 Select finalized columns and write to documents

In [ ]:
# 去掉判断、初步输出列
final_df = filtered_df.drop(columns=['OB',"Extracted"])

#final_df.to_csv(output_path, index=False)
final_df.to_excel(output_path1, index=False)